In [2]:
import pandas as pd
import networkx as nx

In [3]:
airports_df = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
    names=['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude',
        'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source'], index_col=0)

airlines_df = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat',
    names=['Airline ID', 'Name', 'Alias', 'IATA', 'ICAO', 'Callsign', 'Country', 'Active'], index_col=0)

routes_df = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat',
    names=['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport',
        'Destination airport ID', 'Codeshare', 'Stops', 'Equipment'])

planes_df = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat',
    names=['Name', 'IATA code', 'ICAO code'])

In [4]:
airports_df = airports_df.replace({r'\N': None})

airlines_df = airlines_df.fillna('')
airlines_df = airlines_df.replace({r'\N': None})

routes_df = routes_df.fillna('')

In [5]:
airports_df

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
Airport ID,,,,,,,,,,,,,
1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14106,Rogachyovo Air Base,Belaya,Russia,None,ULDA,71.616699,52.478298,272,None,None,None,airport,OurAirports
14107,Ulan-Ude East Airport,Ulan Ude,Russia,None,XIUW,51.849998,107.737999,1670,None,None,None,airport,OurAirports
14108,Krechevitsy Air Base,Novgorod,Russia,None,ULLK,58.625000,31.385000,85,None,None,None,airport,OurAirports


In [36]:
routes_df

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,,0,CR2
1,2B,410,ASF,2966,KZN,2990,,0,CR2
2,2B,410,ASF,2966,MRV,2962,,0,CR2
3,2B,410,CEK,2968,KZN,2990,,0,CR2
4,2B,410,CEK,2968,OVB,4078,,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,,0,734
67660,ZM,19016,FRU,2912,DME,4029,,0,734
67661,ZM,19016,FRU,2912,OSS,2913,,0,734


In [17]:
G = nx.Graph()
G.add_nodes_from(airports_df['IATA'])
G.add_edges_from(list(routes_df[['Source airport', 'Destination airport']].to_records(index=False)))


In [28]:
dc = nx.algorithms.centrality.degree_centrality(G)
df = pd.DataFrame([{'Airport': k, 'Degree Centrality': dc[k]} for k in dc]).sort_values('Degree Centrality', ascending=False)
df


,Airport,Degree Centrality
464,AMS,0.039775
298,FRA,0.039134
1056,CDG,0.038492
6046,IST,0.037851
2873,ATL,0.034804
...,...,...
3395,SDX,0.000000
3400,THG,0.000000
3401,FGI,0.000000
3404,MKC,0.000000


In [29]:
cc = nx.closeness_centrality(G)
df = pd.DataFrame([{'Airport': k, 'Closeness Centrality': cc[k]} for k in cc]).sort_values('Closeness Centrality', ascending=False)
df

,Airport,Closeness Centrality
298,FRA,0.218872
1056,CDG,0.217534
412,LHR,0.216414
464,AMS,0.214084
1650,DXB,0.213886
...,...,...
3695,DUE,0.000000
3698,KOU,0.000000
3699,MJL,0.000000
3700,TCH,0.000000


In [30]:
bc = nx.betweenness_centrality(G)
df = pd.DataFrame([{'Airport': k, 'Betweenness Centrality': bc[k]} for k in bc]).sort_values('Betweenness Centrality', ascending=False)
df

,Airport,Betweenness Centrality
2961,ANC,0.021856
2687,LAX,0.019662
1056,CDG,0.018676
1650,DXB,0.017033
298,FRA,0.015404
...,...,...
2650,PRC,0.000000
2649,HNM,0.000000
2648,NSE,0.000000
2647,PBF,0.000000


In [33]:
kc = nx.eigenvector_centrality(G)
df = pd.DataFrame([{'Airport': k, 'Eigenvector Centrality': kc[k]} for k in kc]).sort_values('Eigenvector Centrality', ascending=False)
df

,Airport,Eigenvector Centrality
464,AMS,1.678425e-01
298,FRA,1.663892e-01
1056,CDG,1.594481e-01
304,MUC,1.484981e-01
1174,FCO,1.363201e-01
...,...,...
1696,TIQ,3.326051e-37
4155,WRZ,3.326051e-37
4156,BBM,3.326051e-37
335,BWE,3.326051e-37
